In [ ]:
from termcolor import colored
from itertools import zip_longest
import requests

LINK = "https://raw.githubusercontent.com/ZXLTRXN/5_words_game_cheat/main/rus_words.bin"
ALPHABET = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
PREV_WORDS = ["замок", "танго", "абзац", "обмен", "взнос", "успех"]

# map (position : letter) - yellow background
right_positions = {}

# map (position : list of letters) - white background
wrong_positions = {}

# list of letters on a gray background
banned_letters = []

def read_file(name = "rus_words.bin"):
  with open(name, "rb") as f:
      string = f.read()
      decoded = string.decode()
      return decoded.split()

def read_remote_file(link = LINK):
  f = requests.get(link)
  return f.text.split()

def input_parser(string: str):
  if string == "": raise KeyboardInterrupt()
  exprs = string.split()
  for i, expr in enumerate(exprs):
    if len(expr) == 2 and expr[0] != '!': t = None
    else : t = expr[0]
    if t != None: expr = expr[1:]

    if t not in ('!',"%", None): 
      raise Exception(f"Недопустимый символ типа в выражении {i+1}")
    if not is_russian_letter(expr[0]):
      raise Exception(f"Не буква {expr[0]} в выражении {i+1}")
    if t == '!':
      expr_resolver(t, expr[0], 0)
    else:
      if int(expr[1]) not in range(1, 6): 
        raise Exception(f"Недопустимое число в выражении {i+1}")
      expr_resolver(t, expr[0], int(expr[1]))

def is_russian_letter(text: str, alphabet = ALPHABET):
    return not alphabet.isdisjoint(text.lower())

def expr_resolver(t: str, letter: str, position: int):
  offset_position = position - 1
  if t == None:
    right_positions.update({offset_position : letter})
  elif t == '%':
    wrong_positions.setdefault(offset_position,[]).append(letter)
  elif t == '!':
    banned_letters.append(letter)

def filter_function(word: str):
  if any(l in word for l in banned_letters): return False
  if any(word[k] != right_positions[k] for k in right_positions.keys()): return False
  if any(word[k] in wrong_positions[k] for k in wrong_positions.keys()): return False

  for k in wrong_positions.keys():
    for l in wrong_positions[k]:
      if  l not in word: return False
  return True

def prt(words: list, count: int):
  words = words[:count]
  for w1, w2, w3, w4 in zip_longest(words[0::4], words[1::4], words[2::4], words[3::4], fillvalue = ''):
     print(f"{w1}\t{w2}\t{w3}\t{w4}")

def prt_status():
  print(colored(right_positions, 'yellow'))
  print(wrong_positions)
  print(colored(banned_letters, 'grey'))

def prt_rules():
    print(colored("Далее вводите запрашиваемые буквы через пробел в формате :\n" + 
        "а1 - для желтых букв\n" + 
        "%а2 - для белых\n" +
        "!к - для серых\n" +
        "Например: !с !т %а3 !л !ь", "cyan"))

def main():
  prt_rules()
  dictionary = read_remote_file()
  for i in range(7):
    print(f"Введите полученные буквы для {i+1} слова")
    try:
      input_parser(input())
    except KeyboardInterrupt:
      print(colored("Конец", 'red'))
      break
    except Exception as e: 
      print(colored(f"GG {str(e)}",'red'))
    dictionary = list(filter(filter_function, dictionary))
    prt(dictionary, 60)
    prt_status()

try: 
  main() 
except Exception as e: 
  print(colored("GG текущие значения:",'red'))
  prt_status()
  print(colored(str(e),'red'))


In [ ]:
# https://github.com/Harrix/Russian-Nouns/blob/main/dist/russian_nouns.txt
def make_5_letter_words_file():
  rus_dict = []
  with open("russian_nouns.txt", 'r') as f:
    lines = f.readlines()
    for line in lines:
      word = line.strip()
      if len(word) == 5:
        rus_dict.append(word)

  with open("rus_words.bin", "wb") as f:
    res = ' '.join(rus_dict)
    f.write(res.encode())